In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read, get_array_diff

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Operacionalización de datos

### Alumno desertor
- Requisito: No se debe haber titulado para la carrera analizada

### Tipos de deserción

- Sale del sistema y no vuelve a ingresar
- Sale del sistema y vuelve después de 1, 2 ó 3 años:
    - Vuelve a la misma carrera en la misma institución (no deserta, "congela")
    - Vuelve a otra carrera en la misma institución
    - Vuelve a la misma carrera en otra institución
    - Vuelve a otra carrera en otra institución 

# Matriculados

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

- Se considera todos los matriculados en CFT / IP desde 2015

In [2]:
# Matriculados de CFT / IP
mat_cft_ip = pd.read_csv(f"{BASE}/tmp_data/fechas_titulacion.csv")
mat_cft_ip.head()

,codigo_unico,mrun,fecha_obtencion_titulo
0,I111S2C413J1V1,15015557.0,20220228.0
1,I117S1C60J4V1,6100912.0,20220228.0
2,I260S50C108J1V1,2713742.0,20220228.0
3,I260S38C47J2V1,14478456.0,20220228.0
4,I260S3C280J1V1,23547583.0,20220228.0


In [3]:
mat_cft_ip.isna().sum()

codigo_unico                   0
mrun                           0
fecha_obtencion_titulo    815446
dtype: int64

In [4]:
# Se carga datos de matrícula para estos alumnos, considerando todas las instituciones
cols = [
    'cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 'dur_estudio_carr',
    'cod_inst', 'cod_carrera'
]

mruns = mat_cft_ip["mrun"].unique()

matriculados = pd.concat([
    iter_df_read(f, "mrun", mruns, sep=";", usecols=cols)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).drop_duplicates().reset_index(drop=True)

  0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
# Se agrega columna que indica titulación

if "fecha_obtencion_titulo" not in matriculados.columns:
    len_or = matriculados.shape[0]
    matriculados = pd.merge(
        matriculados,
        mat_cft_ip,
        how="left"
    )
    
    assert matriculados.shape[0] == len_or

## Pre procesamiento: Año de titulación

In [6]:
matriculados["anio_titulacion"] = matriculados["fecha_obtencion_titulo"].fillna("").astype(str).str[:4].replace([""], [np.nan]).astype(float)
matriculados["anio_titulacion"].describe().round()

count    1416585.0
mean        2019.0
std            2.0
min         2015.0
25%         2017.0
50%         2019.0
75%         2020.0
max         2022.0
Name: anio_titulacion, dtype: float64

## Pre prcesamiento: Deserciones

### Primer caso: Sale del sistema y no vuelve a ingresar
- Solo han tomado 1 carrera
- No se han titulado
- Puede o no tener "años en banco"
- No se han matriculado el año 2022 (criterio decisivo)

In [24]:
cft_ip = ["Institutos Profesionales", "Centros de Formación Técnica"]

In [26]:
ruts_matriculados_cft_ip = matriculados[
    matriculados["tipo_inst_1"].isin(cft_ip)
]["mrun"].unique()

print("Cant. personas que ingresaron a estudiar entre 2015 y 2021 y se matricularon en CFT / IP (universo total): ", ruts_matriculados_cft_ip.shape[0])

Cant. personas que ingresaron a estudiar entre 2015 y 2021 y se matricularon en CFT / IP (universo total):  1113239


In [32]:
cant_carreras = matriculados[
    matriculados["mrun"].isin(ruts_matriculados_cft_ip)
][["mrun", "codigo_unico"]].drop_duplicates().groupby(["mrun"]).size().reset_index(name="cant_carreras")

ruts_1_carrera = cant_carreras[cant_carreras["cant_carreras"] == 1]["mrun"].unique()

print("Dentro de los anteriores, cantidad que solo ha tomado 1 carrera: ", ruts_1_carrera.shape[0])

Dentro de los anteriores, cantidad que solo ha tomado 1 carrera:  647706


In [36]:
ruts_no_tit_1_carr = matriculados[
    (matriculados["mrun"].isin(ruts_1_carrera))
    & (matriculados["anio_titulacion"].isna())
]["mrun"].unique()

print("Dentro de los anteriores, no se han titulado: ", ruts_no_tit_1_carr.shape[0])

Dentro de los anteriores, no se han titulado:  327910


In [37]:
ult_mat_no_tit_1_carr = matriculados[
    matriculados["mrun"].isin(ruts_no_tit_1_carr)
].groupby(["mrun"]).agg({
    "cat_periodo": "max"
}).reset_index()

ruts_no_tit_1_carr_no_mat_2022 = ult_mat_no_tit_1_carr[ult_mat_no_tit_1_carr["cat_periodo"] < 2022]["mrun"].unique()

print("De los anteriores, no se matricularon el año 2022: ", ruts_no_tit_1_carr_no_mat_2022.shape[0])

De los anteriores, no se matricularon el año 2022:  309391


In [ ]:
cant_anios_mat = matriculados[
    matriculados["mrun"].isin(ruts_no_tit_1_carr_no_mat_2022)
].groupby(["mrun"]).agg({
    "cat_periodo": pd.Series.nunique
}).reset_index()

ruts_no_tit_1_carr_no_mat_2022_estmatriculados[
    matriculados["mrun"].isin(ruts_no_tit_1_carr_no_mat_2022)
].groupby(["mrun"]).agg({
    "cat_periodo": get_array_diff
}).reset_index()

In [13]:
max_dif_entre_mat = matriculados[
    matriculados["mrun"].isin(ruts_no_tit_1_carr_no_mat_2022)
].groupby(["mrun"]).agg({
    "cat_periodo": get_array_diff
}).reset_index()

ruts_desertores_1 = max_dif_entre_mat[max_dif_entre_mat["cat_periodo"] <= 1]["mrun"].unique()

print(
    f"Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, solo han tomado 1 carrera, no estudian actualmente, y se han matriculado siempre años seguidos: "
    f"{ruts_desertores_1.shape[0]}"
)

Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, solo han tomado 1 carrera, no estudian actualmente, y se han matriculado siempre años seguidos: 299828


In [14]:
print(f"% Desertores Criterio 1: {np.round(ruts_desertores_1.shape[0] / ruts_matriculados.shape[0] * 100, 1)}%")

% Desertores Criterio 1: 26.9%


### Segundo caso:  Sale del sistema y vuelve a la misma carrera en la misma institución
Además se considera que:
- Ha pasado el tiempo de su carrera evaluada (`ruts_matriculados`)
- Se puede haber titulado o no
- Pueden haber estudiado más de 1 carrera
- Pueden o no estar estudiante actualmente (mismo `codigo_unico` u otro)

In [15]:
max_dif_entre_mat_rut_carr = matriculados[
    matriculados["mrun"].isin(ruts_matriculados)
].groupby(["mrun", "codigo_unico"]).agg({
    "cat_periodo": get_array_diff
}).reset_index()

max_dif_entre_mat_rut_carr.head()

,mrun,codigo_unico,cat_periodo
0,5.0,I260S47C118J1V1,0
1,5.0,I374S1C9J1V1,0
2,19.0,I536S0C39J1V1,1
3,37.0,I498S6C132J2V1,1
4,107.0,I111S12C104J1V1,1


In [21]:
ruts_desertores_2_1 = max_dif_entre_mat_rut_carr[max_dif_entre_mat_rut_carr["cat_periodo"] == 2][["mrun", "codigo_unico"]].drop_duplicates()
ruts_desertores_2_2 = max_dif_entre_mat_rut_carr[max_dif_entre_mat_rut_carr["cat_periodo"] == 3][["mrun", "codigo_unico"]].drop_duplicates()
ruts_desertores_2_3 = max_dif_entre_mat_rut_carr[max_dif_entre_mat_rut_carr["cat_periodo"] == 4][["mrun", "codigo_unico"]].drop_duplicates()

print(f"Cant. matriculas de CFT/IP que debieron terminar carrera y congelaron 1 año: ", ruts_desertores_2_1.shape[0])
print(f"Cant. matriculas de CFT/IP que debieron terminar carrera y congelaron 2 años: ", ruts_desertores_2_2.shape[0])
print(f"Cant. matriculas de CFT/IP que debieron terminar carrera y congelaron 3 años: ", ruts_desertores_2_3.shape[0])

Cant. matriculados de CFT/IP que debieron terminar carrera y congelaron 1 año:  44711
Cant. matriculados de CFT/IP que debieron terminar carrera y congelaron 2 años:  10345
Cant. matriculados de CFT/IP que debieron terminar carrera y congelaron 3 años:  3307


### Tercer caso: Vuelve a otra carrera en la misma institución
- Puede o no haber pasado el tiempo de su primera carrera
- No se han titulado de su primera carrera
- Ha tomado al menos 1 carrera más en la misma institución
- Puede o no tener matrícula en el 2022, pero de tener debe ser de otra carrera diferente a la base
- Puede o no tener años en blanco entre la primera carrera y la siguiente
- Puede o no tener años en blanco dentro de cada carrera